In [1]:
import random

import numpy as np
import pandas as pd

from src.dataset import get_dataset
from src.models.elog import ELOgPredictor

In [2]:
ITERATIONS = 1
random.seed(5438)
np.random.seed(5438)

In [3]:
dataset = get_dataset()

/mnt/c/Personal/football-matches-prediction/src/dataset.py:47: DtypeWarning: Columns (11,14,50,102,123,125,129,131,133,135,137,139,141,143,145,147,149,157,158,160,161,163,165,167,169,171,173,184,186,187,205,206) have mixed types. Specify dtype option on import or set low_memory=False.
  mls_df = pd.read_csv("data/mls_matches.csv")


In [4]:
dataset

,team_id,opponent_id,team_at_home,opponent_at_home,team_score,opponent_score,fold
0,Scotland,England,1.0,0.0,0,0,international
1,England,Scotland,1.0,0.0,4,2,international
2,Scotland,England,1.0,0.0,2,1,international
3,England,Scotland,1.0,0.0,2,2,international
4,Scotland,England,1.0,0.0,3,0,international
...,...,...,...,...,...,...,...
143000,Sport Lisboa e Benfica,Sporting Clube de Braga,1.0,0.0,3,0,europe
143001,Panathinaikos Athlitikos Omilos,APS Atromitos Athinon,1.0,0.0,2,1,europe
143002,Fulham Football Club,Watford FC,1.0,0.0,4,1,europe
143003,Panthessalonikios Athlitikos Omilos Konstantin...,Athlitiki Enosi Konstantinoupoleos,1.0,0.0,1,1,europe


In [5]:
model = ELOgPredictor()

In [6]:
X = dataset[["team_id", "opponent_id", "team_at_home", "opponent_at_home"]]
y = dataset[["team_score", "opponent_score"]]

In [7]:
model.fit(X, y)

In [8]:
model.logit.model.predict(
    model.logit.params, exog=pd.DataFrame(data=[{"predicted_score_difference": 0.0}])
)

array([[0.35695543, 0.28608557, 0.356959  ]])

In [9]:
model.predict(
    pd.DataFrame(
        [
            {
                "team_id": "Argentina",
                "opponent_id": "Brazil",
                "team_at_home": 0.0,
                "opponent_at_home": 0.0,
            },
        ]
    )
)

array([[0.35695543, 0.28608557, 0.356959  ]])

In [10]:
model.predict(
    pd.DataFrame(
        [
            {
                "team_id": "Brazil",
                "opponent_id": "Argentina",
                "team_at_home": 0.0,
                "opponent_at_home": 0.0,
            },
        ]
    )
)

array([[0.35695543, 0.28608557, 0.356959  ]])

In [11]:
model.predict_and_update(X, y)

array([[0.42654169, 0.28053235, 0.29292596],
       [0.42858287, 0.28021552, 0.29120161],
       [0.39534216, 0.28433303, 0.3203248 ],
       ...,
       [0.7766678 , 0.14193725, 0.08139495],
       [0.40271483, 0.28361681, 0.31366837],
       [0.83724712, 0.10623802, 0.05651486]], shape=(143005, 3))

In [12]:
model.predict(
    pd.DataFrame(
        [
            {
                "team_id": "Argentina",
                "opponent_id": "Brazil",
                "team_at_home": 0.0,
                "opponent_at_home": 0.0,
            },
        ]
    )
)

array([[0.40166393, 0.28372598, 0.31461009]])

In [13]:
teams_df = dataset[dataset["fold"].isin(["international"])]
teams = set(teams_df["team_id"].unique().tolist()).union(set(teams_df["opponent_id"].unique().tolist()))
ranking = {team: 0 for team in teams}

for team in ranking:
    for opponent in ranking:
        if team == opponent:
            continue
        pred = model.predict(
            pd.DataFrame(
                [
                    {
                        "team_id": team,
                        "opponent_id": opponent,
                        "team_at_home": 1.0,
                        "opponent_at_home": 0.0,
                    },
                ]
            )
        )
        ranking[team] += float(3 * pred[0][0] + 1 * pred[0][1])

In [14]:
pd.DataFrame(list(ranking.items()), columns=['Team', 'Score']).sort_values(by='Score', ascending=False).reset_index(drop=True).head(20)

,Team,Score
0,Argentina,943.873707
1,Spain,942.472546
2,Brazil,933.425644
3,France,920.838017
4,Colombia,917.252774
5,England,901.298180
6,Portugal,900.326214
7,Netherlands,893.994155
8,Germany,893.672171
9,Belgium,892.570673


In [15]:
model.predict(
    pd.DataFrame(
        [
            {
                "team_id": "Argentina",
                "opponent_id": "France",
                "team_at_home": 0.0,
                "opponent_at_home": 0.0,
            },
        ]
    )
)

array([[0.45073684, 0.27627149, 0.27299167]])

In [ ]:
teams_df = dataset[~dataset["fold"].isin(["international"])]
teams = set(teams_df["team_id"].unique().tolist()).union(set(teams_df["opponent_id"].unique().tolist()))
ranking = {team: 0 for team in teams}

for team in ranking:
    for opponent in ranking:
        if team == opponent:
            continue
        pred = model.predict(
            pd.DataFrame(
                [
                    {
                        "team_id": team,
                        "opponent_id": opponent,
                        "team_at_home": 1.0,
                        "opponent_at_home": 0.0,
                    },
                ]
            )
        )
        ranking[team] += float(3 * pred[0][0] + 1 * pred[0][1])

In [ ]:
pd.DataFrame(list(ranking.items()), columns=['Team', 'Score']).sort_values(by='Score', ascending=False).reset_index(drop=True).head(50)